In [1]:
from langchain.schema import AIMessage
from langgraph.checkpoint.memory import InMemorySaver
from config import UserNeeds
from graph import UserNeedGraph


In [2]:
# 初始化 user_needs 实例
user_needs: UserNeeds = {
    "Price_Range_Min": {"value": None, "constraint_type": None, "description": "A integer representing the minimum price."},
    "Price_Range_Max": {"value": None, "constraint_type": None, "description": "A integer representing the maximum price."},
    "Location_MRT": {"value": None, "constraint_type": None, "description": "A string representing the nearest MRT station."},
    "Bedroom_Count": {"value": None, "constraint_type": None, "description": "A integer representing the number of bedrooms."},
}




memory = InMemorySaver()
graph_system = UserNeedGraph()

graph = graph_system.graph_builder.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}



# 第一次初始化 state
state = {"messages": [], "user_needs": user_needs}
graph.update_state(config, state)


while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        break
    
    state["messages"].append({"role": "user", "content": user_input})

    # 运行图
    result = graph.invoke(state, config)

    # 保存 state
    graph.update_state(config, result)

    # 打印 AI 回复
    ai_message = result["messages"][-1]
    if isinstance(ai_message, AIMessage):
        print(f"AI: {ai_message.content}")

    # 打印当前 user_needs
    print("-----")
    print("    当前用户需求：")
    for k, v in result["user_needs"].items():
        print(f"      - {k}: {v['value']}")
    print("-----")

    if all(v["value"] is not None for v in result["user_needs"].values()):
        print("✅ 所有需求已收集完成！")
        break

AI: OK. I will need to know your desired price range and how many bedrooms you need.
-----
    当前用户需求：
      - Price_Range_Min: None
      - Price_Range_Max: None
      - Location_MRT: None
      - Bedroom_Count: None
-----
